## Per-Metric Barplots Across Different Inputs (Per Model)

# Library requirements

The libraries used in this colab are standard for [colab.research.google.com](https://colab.research.google.com)

In case of import failures, manually install the failing packages
by running `!pip install <package>` in a new cell.

---
#### NOTE: MAKE SURE YOU CHANGE YOUR RUNTIME TYPE TO T4 GPU


# Run required imports

In [ ]:
import matplotlib as mpl
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
from matplotlib.cm import ScalarMappable

from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt


import numpy as np
import os
import pandas as pd
import pickle

from scipy import stats
from scipy.stats import ranksums, ttest_rel

import shap

from statsmodels.stats.multitest import multipletests

import seaborn as sns

from sklearn import metrics, model_selection
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.multitest import multipletests

import tensorflow as tf

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import umap

import xgboost

In [ ]:
# Colab specifics
from google.colab import files
from google.colab import sheets

import warnings
warnings.filterwarnings('ignore')

# Define paths & upload data

File uploads and algorithm outputs can be explored under the "Files" tab in the menu on the left.

In [ ]:
INPUTS_FOLDER = '/content/input_data'
OUTPUTS_FOLDER = '/content/output_data'

DATA_SOURCE_PATH = os.path.join(INPUTS_FOLDER,  "data.pq")

KFOLD_RESULTS_SAE_PATH = os.path.join(INPUTS_FOLDER, "simple_autoencoder_kfold_latest.pq")
KFOLD_RESULTS_MAE_PATH = os.path.join(INPUTS_FOLDER, "masked_autoencoder_kfold_latest.pq")

LOOCV_RESULTS_XGB_PATH = os.path.join(INPUTS_FOLDER, "xgboost_loocv_latest.pq")
KFOLD_RESULTS_XGB_PATH = os.path.join(INPUTS_FOLDER, "xgboost_kfold_latest.pq")

XGBOOST_STUDY_RESULTS_PATH = os.path.join(INPUTS_FOLDER, 'xgboost_ablation_study_results.zip')
XGBOOST_STUDY_RESULTS_FOLDER = os.path.join(OUTPUTS_FOLDER, 'xgboost_ablation_study_results')

AUTOENCODER_STUDY_RESULTS_PATH = os.path.join(INPUTS_FOLDER, 'autoencoder_ablation_study_results.zip')
AUTOENCODER_STUDY_RESULTS_FOLDER = os.path.join(OUTPUTS_FOLDER, 'autoencoder_ablation_study_results')

path_list = [DATA_SOURCE_PATH, KFOLD_RESULTS_SAE_PATH, KFOLD_RESULTS_MAE_PATH, LOOCV_RESULTS_XGB_PATH, KFOLD_RESULTS_XGB_PATH, XGBOOST_STUDY_RESULTS_PATH, AUTOENCODER_STUDY_RESULTS_PATH]
folder_list = [INPUTS_FOLDER, OUTPUTS_FOLDER, AUTOENCODER_STUDY_RESULTS_FOLDER]

In [ ]:
for folder in folder_list:
  if not os.path.exists(folder):
    os.makedirs(folder)

### Upload files from your filesystem

This section expects that files downloaded from previous colabs were not renamed.

The following files are expected to be uploaded:
* `data.pq`
* `simple_autoencoder_kfold_latest.pq`
* `masked_autoencoder_kfold_latest`
* `xgboost_loocv_latest.pq`
* `xgboost_kfold_latest.pq`
* `xgboost_study_results.zip` - optional, file is large
* `autoencoder_study_results.zip`

In [ ]:
# @title Upload files from your filesystem
uploaded = files.upload(target_dir=INPUTS_FOLDER)

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes to folder {folder}'.format(
      name=fn, length=len(uploaded[fn]), folder=INPUTS_FOLDER))

for fn in uploaded.keys():
  if fn not in path_list:
    raise ValueError(f"Wrong files uploaded: {fn}. Re-upload or use manual upload and adjust paths yourself.")

Saving autoencoder_ablation_study_results.zip to /content/input_data/autoencoder_ablation_study_results.zip
Saving data.pq to /content/input_data/data.pq
Saving masked_autoencoder_kfold_latest.pq to /content/input_data/masked_autoencoder_kfold_latest.pq
Saving simple_autoencoder_kfold_latest.pq to /content/input_data/simple_autoencoder_kfold_latest.pq
Saving xgboost_kfold_latest.pq to /content/input_data/xgboost_kfold_latest.pq
Saving xgboost_loocv_latest.pq to /content/input_data/xgboost_loocv_latest.pq
User uploaded file "/content/input_data/autoencoder_ablation_study_results.zip" with length 6893909 bytes to folder /content/input_data
User uploaded file "/content/input_data/data.pq" with length 423429 bytes to folder /content/input_data
User uploaded file "/content/input_data/masked_autoencoder_kfold_latest.pq" with length 25170 bytes to folder /content/input_data
User uploaded file "/content/input_data/simple_autoencoder_kfold_latest.pq" with length 25170 bytes to folder /content/i

In [ ]:
# @title Unzip archives
# Uncomment if you have uploaded xgboost_study_results.zip as well
# !unzip $XGBOOST_STUDY_RESULTS_PATH
!unzip $AUTOENCODER_STUDY_RESULTS_PATH

Archive:  /content/input_data/autoencoder_ablation_study_results.zip
  inflating: output_data/autoencoder_ablation_study_results/simple_autoencoder_kfold_latest.pq  
  inflating: output_data/autoencoder_ablation_study_results/masked_autoencoder_kfold_latest.pq  
   creating: output_data/autoencoder_ablation_study_results/ablation_models/
   creating: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/
  inflating: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_iteration_4_simple_autoencoder_EXP_2_n_days_120.pth  
  inflating: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_iteration_3_simple_autoencoder_EXP_2_n_days_120.pth  
  inflating: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_iteration_2_simple_autoencoder_EXP_2_n_days_120.pth  
  infla

# Helper classes and functions

In [ ]:
# @title Bar Chart Utility Function

def plot_nature_style_bar_chart(input_groups,
                                classification_metric_avg,
                                classification_metric_std,
                                metric_name="",
                                color = '#66c2a5',
                                bar_width=0.30,
                                figsize=(28, 5),
                                pixels_per_inch=720):
  """Generates a bar chart with a clean, Nature-inspired style for
  """

  # Data for the plot
  num_groups = len(input_groups)
  index = np.arange(num_groups)

  # Create the figure and axes object
  _, ax = plt.subplots(figsize=figsize, dpi=pixels_per_inch)

  # --- Nature-Inspired Styling ---
  ax.spines['top'].set_linewidth(2)
  ax.spines['right'].set_linewidth(2)
  ax.spines['bottom'].set_linewidth(2)
  ax.spines['left'].set_linewidth(2)
  ax.tick_params(axis='both', which='major', labelsize=10, direction='out', length=4, width=0.5)
  _ = ax.bar(index, classification_metric_avg, bar_width, label=metric_name, color=color,
                 yerr=classification_metric_std, capsize=10, error_kw={'elinewidth': 2}, linewidth=2, edgecolor="black",
                 )

  ax.set_xlabel('Input Feature Set\n \n', fontsize=22, fontweight='bold')
  ax.set_ylabel(f'{metric_name}\n', fontsize=22, fontweight='bold')
  ax.set_xticks(index)
  ax.set_xticklabels(input_groups, rotation=55, ha='right', fontdict={'fontsize': 12, 'fontweight': 'bold'})
  ax.set_yticklabels([f'{tick:.1f}' for tick in ax.get_yticks()], fontdict={'fontsize': 12, 'fontweight': 'bold'})
  ax.legend(loc='best', ncol=3, fontsize=16)
  # Grid (subtle)
  ax.grid(axis='y', linestyle='--', linewidth=2, alpha=0.3, zorder=1)

  plt.tight_layout()
  plt.show()

In [ ]:
# @title Point Plots with Error Bars Utility Functions

def plot_nature_style_point_chart(input_groups,
                                classification_metric_avg,
                                classification_metric_std,
                                metric_name="",
                                color='#66c2a5',
                                marker='o',
                                markersize=10,
                                figsize=(28, 5),
                                pixels_per_inch=720):
  """Generates a point chart with error bars in a clean, Nature-inspired style.

  This function replaces bar plots with individual data points and error bars
  to represent the mean and standard deviation of a given metric across
  different input groups.

  Args:
    input_groups (list): A list of strings representing the names of the input
      feature sets.
    classification_metric_avg (list): A list of floats representing the average
      values of the classification metric for each input group.
    classification_metric_std (list): A list of floats representing the
      standard deviations of the classification metric for each input group.
    metric_name (str, optional): The name of the classification metric being
      plotted (e.g., "Accuracy", "AUC"). Defaults to "".
    color (str, optional): The color of the markers and error bars. Defaults to
      '#66c2a5'.
    marker (str, optional): The shape of the marker used for data points.
      Defaults to 'o' (circle).
    markersize (int, optional): The size of the markers. Defaults to 10.
    figsize (tuple, optional): The size of the figure in inches (width, height).
      Defaults to (28, 5).
    pixels_per_inch (int, optional): The resolution of the figure in pixels per
      inch. Defaults to 720.
  """

  num_groups = len(input_groups)
  index = np.arange(num_groups)

  _, ax = plt.subplots(figsize=figsize, dpi=pixels_per_inch)

  # --- Nature-Inspired Styling ---
  ax.spines['top'].set_linewidth(2)
  ax.spines['right'].set_linewidth(2)
  ax.spines['bottom'].set_linewidth(2)
  ax.spines['left'].set_linewidth(2)
  ax.tick_params(axis='both',
                 which='major',
                 labelsize=10,
                 direction='out',
                 length=4, width=0.5)

  # Plot points with error bars
  ax.errorbar(index, classification_metric_avg, yerr=classification_metric_std,
              fmt=marker, markersize=markersize, capsize=18, color=color,
              linewidth=2, elinewidth=5, ecolor=color,
              markeredgecolor='black', markeredgewidth=2)

  # ax.set_xlabel('Input Feature Set\n \n', fontsize=22, fontweight='bold')
  ax.set_ylabel(f'{metric_name}\n', fontsize=22, fontweight='bold')
  ax.set_xticks(index)
  ax.set_xticklabels(input_groups, rotation=20, ha='right', fontdict={'fontsize': 22, 'fontweight': 'bold'})
  ax.set_yticklabels([f'{tick:.2f}' for tick in ax.get_yticks()],
                     fontdict={'fontsize': 22, 'fontweight': 'bold'})
  # Grid (subtle)
  ax.grid(axis='y', linestyle='--', linewidth=2, alpha=0.3, zorder=1)

  plt.tight_layout()
  plt.show()

In [ ]:
# @title Data Processing Utils
def preprocess_data(X_train, X_test, scaler_name="standard"):
    """
    Preprocesses and splits a Pandas DataFrame for neural network training,
    returning the scaler object for inverse transformation.

    Args:
        df: The input Pandas DataFrame.
        target_cols: A list of column names representing the target variables (optional).
                     If not provided, the entire DataFrame is considered for normalization.

    Returns:
        A tuple containing:
            - Train and test DataFrames, potentially with normalized values and split into
              features (X) and targets (y) if target_cols are provided.
            - The fitted StandardScaler object for inverse transformation.
    """
    if scaler_name == "minmax":
      scaler = MinMaxScaler()
    elif scaler_name == "standard":
      scaler = StandardScaler()
    else:
      raise ValueError(f"Unknown scaler name: {scaler_name}")

    x_train_normalized = scaler.fit_transform(X_train)
    x_test_normalized = scaler.transform(X_test)

    return x_train_normalized, x_test_normalized

# Plots

### Read-in the Latest Results

In [ ]:
with open(KFOLD_RESULTS_MAE_PATH, "rb+") as fd:
  kfold_results_mae = pd.read_parquet(fd)

with open(KFOLD_RESULTS_XGB_PATH, "rb+") as fd:
  kfold_results_xgb = pd.read_parquet(fd)

with open(DATA_SOURCE_PATH, 'rb') as infile:
  training_data_aggregated_all_days = pd.read_parquet(infile)

In [ ]:
kfold_results_mae['experiment_id'] = kfold_results_mae['experiment_name'].apply(lambda x: x.get('name'))
kfold_results_xgb['experiment_id'] = kfold_results_xgb['experiment_name'].apply(lambda x: x.get('name'))

In [ ]:
# 120 days indicates all available data from users

seven_day_subset_mae = kfold_results_mae[kfold_results_mae['n_days']==120]
seven_day_subset_both_xgb = kfold_results_xgb[kfold_results_xgb['n_days']==120]
seven_day_subset_xgb = seven_day_subset_both_xgb[seven_day_subset_both_xgb['learner']=='tree']
seven_day_subset_linear = seven_day_subset_both_xgb[seven_day_subset_both_xgb['learner']=='l1-l2']

len(seven_day_subset_xgb)

28

In [ ]:
# @title Feature Set Names for Plotting

exp_id_to_name_dict = {"EXP_26": "Fasting Glucose +\n Fasting Insulin \n(Trivial)",
                       "EXP_25": "Fasting Insulin \n(Control)",
                       "EXP_9": "Fasting Glucose",
                       "EXP_7": "HbA1c",
                       "EXP_19": "Lipid Panel",
                       "EXP_20": "Metabolic Panel",
                       "EXP_17": "Lipid Panel \n+ Metabolic Panel",
                       "EXP_4": "Demo.",
                       "EXP_6": "Demo.+ HbA1c",
                       "EXP_8": "Demo. + Glucose",
                       "EXP_12": "Demo. + Lipid Panel \n+ Metabolic Panel",
                       "EXP_13": "Demo. + Hypertension",
                       "EXP_23": "Demo. + Lipid Panel",
                       "EXP_24": "Demo. + Metabolic Panel",
                       "EXP_3": "Wearables Subset",
                       "EXP_1": "Wearables + Demo.",
                       "EXP_2": "Fitbit Subset + Demo.",
                       "EXP_5": "Wearables + Demo. \n+ HbA1c",
                       "EXP_10": "Wearables + Demo. \n + Glucose",
                       "EXP_11": "Wearables + Demo. \n+ Lipid Panel \n+ Metabolic Panel",
                       "EXP_14": "Wearables + Demo.\n + Hypertension",
                       "EXP_15": "Wearables + Demo. \n+ Glucose \n+ Hypertension",
                       "EXP_16": "Wearables + Demo. \n+ Lipid Panel \n+ Metabolic Panel \n+ Hypertension",
                       "EXP_18": "Wearables + Demo. \n+ Lipid Panel \n+ Metabolic Panel \n+ HbA1c",
                       "EXP_21": "Wearables + Demo. \n+Lipid Panel",
                       "EXP_22": "Wearables + Demo. \n+ Metabolic Panel",
                       "EXP_27": "Wearables + Demo. \n+ Glucose \n+ Lipid Panel",
                       "EXP_28": "Wearables",
                       }

In [ ]:
# Desired order of experiment_id
order = ["EXP_28", "EXP_4", "EXP_7", "EXP_9", "EXP_19", "EXP_20", "EXP_1",
         "EXP_21", "EXP_10", "EXP_11"][::-1]

# Create a categorical dtype with the desired order
seven_day_subset_xgb['experiment_id'] = pd.Categorical(seven_day_subset_xgb['experiment_id'],
                                                       categories=order,
                                                       ordered=True)

seven_day_subset_mae['experiment_id'] = pd.Categorical(seven_day_subset_mae['experiment_id'],
    categories=order,
    ordered=True)

seven_day_subset_linear['experiment_id'] = pd.Categorical(seven_day_subset_linear['experiment_id'],
    categories=order,
    ordered=True)

# # Sort the DataFrame based on the categorical column
df_for_plotting_xgb = seven_day_subset_xgb.sort_values('experiment_id')
df_for_plotting_mae = seven_day_subset_mae.sort_values('experiment_id')
df_for_plotting_linear = seven_day_subset_linear.sort_values('experiment_id')

# Reset the index if desired

df_for_plotting_xgb = df_for_plotting_xgb.reset_index(drop=True).dropna()
df_for_plotting_mae = df_for_plotting_mae.reset_index(drop=True).dropna()
df_for_plotting_linear = df_for_plotting_linear.reset_index(drop=True).dropna()

df_for_plotting_mae

,n_subjects,n_iterations,saved_model_dir,rl-model,regression_model,learner,latent_space,n_days,experiment_name,type_of_cross_validation,...,auroc_std,auprc_avg,auprc_std,booster,learning_rate,n_estimators,eta,reg_alpha,reg_lambda,experiment_id
0,1159,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,{'description': 'Fitbit + Demographics + Lipid...,kfold,...,0.023270,0.729454,0.030124,gblinear,0.19,50,0.1,0,0,EXP_11
1,1168,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,{'description': 'Fitbit + Demographics + Fasti...,kfold,...,0.030574,0.690977,0.038437,gblinear,0.19,50,0.1,0,0,EXP_10
2,1160,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,{'description': 'Fitbit + Demographics + Lipid...,kfold,...,0.015413,0.669066,0.008682,gblinear,0.19,50,0.1,0,0,EXP_21
3,1168,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Fitbit + Demographics', 'feat...",kfold,...,0.018482,0.628732,0.033118,gblinear,0.19,50,0.1,0,0,EXP_1
4,1172,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Metabolic Panel', 'features':...",kfold,...,0.038574,0.664790,0.041102,gblinear,0.19,50,0.1,0,0,EXP_20
5,1165,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Lipid Panel', 'features': ['p...",kfold,...,0.026678,0.594987,0.037722,gblinear,0.19,50,0.1,0,0,EXP_19
6,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Fasting Glucose', 'features':...",kfold,...,0.034454,0.610108,0.037161,gblinear,0.19,50,0.1,0,0,EXP_9
7,1169,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'HbA1c', 'features': ['partici...",kfold,...,0.030284,0.552963,0.045214,gblinear,0.19,50,0.1,0,0,EXP_7
8,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Demographics', 'features': ['...",kfold,...,0.020240,0.610805,0.014241,gblinear,0.19,50,0.1,0,0,EXP_4
9,1168,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'description': 'Fitbit', 'features': ['partic...",kfold,...,0.019307,0.524594,0.039407,gblinear,0.19,50,0.1,0,0,EXP_28


In [ ]:
feature_set_name = []
r2_scores = []
sensitivity_mean = []
sensitivity_std = []
specificity_mean = []
specificity_std = []
precision_mean = []
precision_std = []
auc_mean = []
auc_std = []
auprc_mean = []
auprc_std = []
exp_name_to_num_subject = {}

for _, row in df_for_plotting_mae.iloc[::-1,:].iterrows():
  if row['experiment_id']== "EXP_26" or row['experiment_id']== "EXP_25":
    continue

  dictionary_key = row['experiment_id']
  if dictionary_key is np.nan:
    dictionary_key="EXP_28"

  feature_set_name.append(exp_id_to_name_dict[dictionary_key])
  r2_scores.append(row['R2_score'])
  sensitivity_mean.append(row['sensitivity_avg'])
  sensitivity_std.append(row['sensitivity_std'])
  specificity_mean.append(row['specificity_mean'])
  specificity_std.append(row['specificity_std'])
  precision_mean.append(row['precision_avg'])
  precision_std.append(row['precision_std'])
  auc_mean.append(row['auroc_avg'])
  auc_std.append(row['auroc_std'])
  auprc_mean.append(row['auprc_avg'])
  auprc_std.append(row['auprc_std'])
  exp_name_to_num_subject[exp_id_to_name_dict[dictionary_key]] = row['n_subjects']

In [ ]:
metrics_names = ["AUROC", "Sensitivity", "Specificity", "Precision", "AUPRC"]
metric_values = [(auc_mean, auc_std),
                 (sensitivity_mean, sensitivity_std),
                 (specificity_mean, specificity_std),
                 (precision_mean, precision_std),
                 (auprc_mean, auprc_std)]

for metric_index in range(len(metrics_names)):
  metric_name = metrics_names[metric_index]
  metric_avg, metric_std = metric_values[metric_index]

  plot_nature_style_point_chart(feature_set_name,
                              metric_avg,
                              metric_std,
                              metric_name=metric_name,
                              color =sns.color_palette("blend:#7AB,#EDA")[metric_index],
                              markersize=40,
                              figsize=(40, 8),
                              pixels_per_inch=720)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Feature Set Names for Plotting

exp_id_to_name_dict = {"EXP_26": "Fasting Glucose + Fasting Insulin (Trivial)",
                       "EXP_25": "Fasting Insulin (Control)",
                       "EXP_9": "Fasting Glucose",
                       "EXP_7": "HbA1c",
                       "EXP_19": "Lipid Panel",
                       "EXP_20": "Metabolic Panel",
                       "EXP_17": "Lipid Panel + Metabolic Panel",
                       "EXP_4": "Demo.",
                       "EXP_6": "Demo.+ HbA1c",
                       "EXP_8": "Demo. + Glucose",
                       "EXP_12": "Demo. + Lipid Panel + Metabolic Panel",
                       "EXP_13": "Demo. + Hypertension",
                       "EXP_23": "Demo. + Lipid Panel",
                       "EXP_24": "Demo. + Metabolic Panel",
                       "EXP_3": "Wearables Subset",
                       "EXP_1": "Wearables + Demo.",
                       "EXP_2": "Fitbit Subset + Demo.",
                       "EXP_5": "Wearables + Demo. + HbA1c",
                       "EXP_10": "Wearables + Demo. + Glucose",
                       "EXP_11": "Wearables + Demo. + \n Lipid Panel + Metabolic Panel",
                       "EXP_14": "Wearables + Demo. + Hypertension",
                       "EXP_15": "Wearables + Demo. + Glucose + Hypertension",
                       "EXP_16": "Wearables + Demo. + Lipid Panel +\n Metabolic Panel + Hypertension",
                       "EXP_18": "Wearables + Demo. + Lipid Panel +\n Metabolic Panel + HbA1c",
                       "EXP_21": "Wearables + Demo. +Lipid Panel",
                       "EXP_22": "Wearables + Demo. + Metabolic Panel",
                       "EXP_27": "Wearables + Demo. + Glucose + Lipid Panel",
                       "EXP_28": "Wearables",
                       }

In [ ]:
# Desired order of experiment_id

kfold_results_mae['experiment_id'] = kfold_results_mae['experiment_name'].apply(lambda x: x.get('name'))
kfold_results_xgb['experiment_id'] = kfold_results_xgb['experiment_name'].apply(lambda x: x.get('name'))

seven_day_subset_mae = kfold_results_mae[kfold_results_mae['n_days']==120]
seven_day_subset_both_xgb = kfold_results_xgb[kfold_results_xgb['n_days']==120]
seven_day_subset_xgb = seven_day_subset_both_xgb[seven_day_subset_both_xgb['learner']=='tree']
seven_day_subset_linear = seven_day_subset_both_xgb[seven_day_subset_both_xgb['learner']=='l1-l2']

order = ["EXP_28", "EXP_4", "EXP_7", "EXP_9", "EXP_19", "EXP_20", "EXP_1",
         "EXP_21", "EXP_10", "EXP_11", ][::-1]

# Create a categorical dtype with the desired order
seven_day_subset_xgb['experiment_id'] = pd.Categorical(seven_day_subset_xgb['experiment_id'],
                                                       categories=order,
                                                       ordered=True)

seven_day_subset_mae['experiment_id'] = pd.Categorical(seven_day_subset_mae['experiment_id'],
                                                       categories=order,
                                                       ordered=True)

seven_day_subset_linear['experiment_id'] = pd.Categorical(seven_day_subset_linear['experiment_id'],
                                                       categories=order,
                                                       ordered=True)

# Sort the DataFrame based on the categorical column
xgb_results_for_plotting = seven_day_subset_xgb.sort_values('experiment_id')
mae_results_for_plotting = seven_day_subset_mae.sort_values('experiment_id')
linear_results_for_plotting = seven_day_subset_linear.sort_values('experiment_id')
linear_results_for_plotting['max_depth'] = 1

# Reset the index if desired
xgb_results_for_plotting = xgb_results_for_plotting.reset_index(drop=True).dropna()
mae_results_for_plotting = mae_results_for_plotting.reset_index(drop=True).dropna()
linear_results_for_plotting = linear_results_for_plotting.reset_index(drop=True).dropna()

In [ ]:
linear_results_for_plotting

,n_subjects,n_iterations,saved_model_dir,model,learner,n_days,experiment_name,type_of_cross_validation,R2_score,MAE_mean,...,auroc_std,auprc_avg,auprc_std,booster,learning_rate,n_estimators,reg_alpha,reg_lambda,max_depth,experiment_id
0,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,{'description': 'Fitbit + Demographics + Lipid...,kfold,0.479761,0.980527,...,0.052124,0.718350,0.058465,gblinear,0.5,100,0,0,1,EXP_11
1,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,{'description': 'Fitbit + Demographics + Fasti...,kfold,0.448305,1.027256,...,0.030464,0.701181,0.032543,gblinear,0.5,100,0,0,1,EXP_10
2,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,{'description': 'Fitbit + Demographics + Lipid...,kfold,0.299910,1.161867,...,0.018584,0.668122,0.021078,gblinear,0.5,100,0,0,1,EXP_21
3,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Fitbit + Demographics', 'feat...",kfold,0.229795,1.239281,...,0.010793,0.621129,0.025792,gblinear,0.5,100,0,0,1,EXP_1
4,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Metabolic Panel', 'features':...",kfold,0.396550,1.066850,...,0.052777,0.660366,0.057619,gblinear,0.5,100,0,0,1,EXP_20
5,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Lipid Panel', 'features': ['p...",kfold,0.158036,1.263335,...,0.015632,0.562693,0.034524,gblinear,0.5,100,0,0,1,EXP_19
6,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Fasting Glucose', 'features':...",kfold,0.315578,1.163114,...,0.034994,0.609395,0.036969,gblinear,0.5,100,0,0,1,EXP_9
7,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'HbA1c', 'features': ['partici...",kfold,0.200366,1.279926,...,0.036275,0.542409,0.053255,gblinear,0.5,100,0,0,1,EXP_7
8,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Demographics', 'features': ['...",kfold,0.184997,1.259909,...,0.018930,0.589071,0.023336,gblinear,0.5,100,0,0,1,EXP_4
9,1173,5,/content/output_data/xgboost_ablation_study_re...,xgboost,l1-l2,120,"{'description': 'Fitbit', 'features': ['partic...",kfold,0.122205,1.348398,...,0.016274,0.521500,0.033204,gblinear,0.5,100,0,0,1,EXP_28


In [ ]:
feature_set_name = []
r2_scores_mae = []
r2_scores_xgb = []
r2_scores_linear = []

for _, row in xgb_results_for_plotting.iterrows():
  if row['experiment_id']== "EXP_26" or row['experiment_id']== "EXP_25":
    continue
  feature_set_name.append(exp_id_to_name_dict[row['experiment_id']])
  r2_scores_xgb.append(row['R2_score'])

for _, row in mae_results_for_plotting.iterrows():
  if row['experiment_id']== "EXP_26" or row['experiment_id']== "EXP_25":
    continue
  r2_scores_mae.append(row['R2_score'])

for _, row in linear_results_for_plotting.iterrows():
  if row['experiment_id']== "EXP_26" or row['experiment_id']== "EXP_25":
    continue
  r2_scores_linear.append(row['R2_score'])

assert len(feature_set_name) == len(r2_scores_mae) == len(r2_scores_xgb) == len(r2_scores_linear)

In [ ]:
# --- 1. Generate Example Data (replace with your actual data) ---
np.random.seed(42)
num_feature_sets = len(order)
feature_set_names = feature_set_name
r2_linear = np.array(r2_scores_linear)
r2_xgboost = np.array(r2_scores_xgb)
r2_mae_xgboost = np.array(r2_scores_mae)

# --- 2. Create the Plot ---
plt.figure(figsize=(24, 10), dpi=720)

# Scatter plots with connections (using a loop for efficiency)
model_data = [r2_linear, r2_xgboost, r2_mae_xgboost]
model_labels = ["L1-L2 Learners", "Tree Learners", "MAE + L1-L2 Learners"]
model_colors = ['#66c2a5', '#fc8d62', '#8da0cf'] # Distinct color for each model
model_markers = ['o', 's', 'h']  # Different marker for each model

y_positions = np.arange(num_feature_sets)
for i in range(num_feature_sets):
    # Connect the three markers for each feature set
    plt.plot([r2_linear[i], r2_xgboost[i], r2_mae_xgboost[i]], [y_positions[i]] * 3,
             color='k', linestyle='-', linewidth=3, zorder=1)

for model_idx, r2_values in enumerate(model_data):
    for i in range(num_feature_sets):
        plt.plot([r2_values[i], r2_values[i]], [i - 0.2 + (model_idx * 0.2), i - 0.2 + (model_idx * 0.2)],
                 color='gray', linestyle='-', linewidth=8, zorder=1)
    plt.scatter(r2_values, np.arange(num_feature_sets),
                marker=model_markers[model_idx], color=model_colors[model_idx],
                s=400, label=model_labels[model_idx], zorder=2, alpha=0.6)

# --- 3. Enhancements ---
plt.xticks(np.arange(0, 0.6, 0.05), fontsize=15)
plt.yticks(np.arange(num_feature_sets), feature_set_names, fontsize=15)
plt.xlabel("\nR^2 Value", fontsize=20, fontweight='bold')
plt.ylabel("Input Feature Set\n", fontsize=25, fontweight='bold')
plt.title("Comparison of Regression Performance (R^2) Across Models and Input Feature Sets",
          fontsize=22, fontweight='bold')
plt.legend(loc="best", fontsize=16)
plt.xticks(fontsize=16,fontweight="bold")
plt.yticks(fontsize=16,fontweight="bold")
plt.grid(axis='x', linestyle='--', linewidth=1, alpha=0.3)
plt.tight_layout()
plt.show()

## Regression Plots for MAE + XGBoost

In [ ]:
# @title Reading in Our Data (Averaged Over All Days)
with open(DATA_SOURCE_PATH, "rb") as infile:
  training_data_aggregated_all_days = pd.read_parquet(infile)

training_data_aggregated_all_days['hypertension_binary']=(
                  training_data_aggregated_all_days['hypertension'].astype(int))

In [ ]:
training_data_aggregated_all_days

,participant_id,RHR_FreeLiving_mean,RHR_FreeLiving_median,RHR_FreeLiving_std,HRV_FreeLiving_mean,HRV_FreeLiving_median,HRV_FreeLiving_std,STEPS_Daily_mean,STEPS_Daily_median,STEPS_Daily_std,...,framingham_risk_category,optimal_framingham_risk,framingham30_risk,framingham30_risk_category,optimal_framingham30_risk,ascvd_risk,ascvd_risk_category,optimal_ascvd_risk,apri,hypertension_binary
0,12591,65.509615,65.0,1.960948,25.452000,25.0305,4.018824,7329.278846,6557.5,3087.041901,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.350877,1
1,12597,49.201923,49.0,1.623379,15.998155,15.9660,2.052595,5357.019231,4956.0,2354.257334,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.503513,1
2,12599,65.168421,64.0,4.636302,42.677238,43.4890,10.082448,9036.231579,8172.0,4653.169485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.214918,0
3,12612,71.080000,71.0,1.796068,44.996636,46.0220,7.762804,10998.481481,9806.0,5084.491732,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.220629,0
4,12614,65.200000,64.0,4.300286,22.555556,22.2240,5.748754,5953.837438,5469.0,3471.700982,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.612245,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,199756,68.432099,69.0,3.625096,22.274354,22.0030,6.682359,7019.839506,7004.0,2800.065485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.171086,0
1169,199757,65.696078,65.0,3.522732,11.017429,10.9355,1.574011,11991.941176,10074.0,7029.545331,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.249653,1
1170,199758,64.527132,65.0,2.334240,28.951650,27.5440,7.033047,5076.480620,5088.0,2297.668607,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.151313,0
1171,199759,69.549451,69.0,3.029373,14.461491,14.5800,3.405676,5321.453608,4643.0,4216.633141,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.234962,0


In [ ]:
class MaskedAutoencoder(nn.Module):
  """
  Masked autoencoder model with biomarker embeddings.
  """

  def __init__(self, input_dim, latent_space: int = 32):
    super().__init__()
    first_layer_dim= max(2*input_dim, 64)
    self.encoder = nn.Sequential(
        nn.Linear(input_dim, first_layer_dim),
        nn.LeakyReLU(),
        nn.Linear(first_layer_dim, latent_space)
    )
    self.decoder = nn.Sequential(
        nn.Linear(latent_space, input_dim), nn.LeakyReLU()
    )

  def forward(self, x, mask=None, return_encoded=False):
    if mask is None:
      mask = torch.ones_like(x)

    masked_embeddings = x * mask

    # Encode and decode
    encoded = self.encoder(masked_embeddings)
    if return_encoded:
      return encoded, self.decoder(encoded)

    return self.decoder(encoded)

In [ ]:
# @title Scatter Plots with Highlighted Quadrants and Counts

def plot_with_counts_on_quadrants(true_values, predicted_values,
                                  insulin_resistance_threshold:float=2.9,
                                  insulin_sensitive_threshold:float=1.5,
                                  current_r2_score:float=NotImplementedError):
  if current_r2_score is None:
    current_r2_score = metrics.r2_score(true_values, predicted_values)
  else:
    r2 = current_r2_score
  insulin_sensitive_threshold = insulin_sensitive_threshold
  insulin_resistance_threshold = insulin_resistance_threshold

  # Create the plot
  fig, ax = plt.subplots(figsize=(12, 10), dpi=720)

  # Scatter plot
  sns.scatterplot(x=true_values, y=predicted_values, s=70, alpha=0.4,
                  color='blue', edgecolor='black', linewidth=0.5)

  # --- Shade the regions ---
  quadrant_colors = ['rosybrown', # IS-IR
                     'w',
                     'lightgreen', # IS-IS
                     'w',
                     'lightgreen', # IM-IM
                     'w', # IS-IM
                     'lightgreen', # IR-IR
                     'w',
                     'rosybrown', # IR-IS
                     ]

  x_ranges = [
      (0, insulin_sensitive_threshold),
      (insulin_sensitive_threshold, insulin_resistance_threshold),
      (insulin_resistance_threshold, 7),
  ]
  y_ranges = [
      (insulin_resistance_threshold, 7),
      (insulin_sensitive_threshold, insulin_resistance_threshold),
      (0, insulin_sensitive_threshold),
  ]
  for i, (x_start, x_end) in enumerate(x_ranges):
      for j, (y_start, y_end) in enumerate(y_ranges):
          ax.add_patch(Rectangle((x_start, y_start), x_end - x_start, y_end - y_start,
                                  color=quadrant_colors[i * 3 + j], alpha=0.3, zorder=0))

  # Add horizontal and vertical lines (on top of the shaded regions)
  plt.axvline(x=insulin_sensitive_threshold, color='orange', linestyle=':',
              linewidth=5, label='Insulin Sensitive Threshold')
  plt.axvline(x=insulin_resistance_threshold, color='red',
              linestyle='--', linewidth=5, label='Insulin Resistance Threshold')
  plt.axhline(y=insulin_sensitive_threshold, color='orange', linestyle=':', linewidth=5)
  plt.axhline(y=insulin_resistance_threshold, color='red', linestyle='--', linewidth=5)
  plt.xlim(0, 7)
  plt.ylim(0, 7)
  # Enhance plot elements
  plt.xlabel("\nTrue HOMA-IR", fontsize=14, fontweight='bold')
  plt.ylabel("Predicted HOMA-IR", fontsize=14, fontweight='bold')
  plt.title(f"{exp_id_to_name_dict[experiment_name['name']]} \n "
            r"($R^2$ ="
            f"{r2:.2f})", fontsize=18, fontweight='bold')
  plt.xticks(fontsize=12)
  plt.yticks(fontsize=12)
  plt.grid(True, linestyle='--', alpha=0.5)
  # Define threshold values

  print(len(true_values))

  quadrant_coordinates = {'IS-IR': (0.1, 6),
                       'IR-IR': (6,6),
                       'IM-IM': (2,2),
                       'IS-IS': (0.1,0.5),
                       'IR-IS': (6, 0.5)}
  quadrant_counts = {
    'IS-IR': ((true_values < insulin_sensitive_threshold) &
           (predicted_values > insulin_resistance_threshold)).sum(),
    'IR-IR': ((true_values >= insulin_resistance_threshold) &
           (predicted_values >= insulin_resistance_threshold)).sum(),
    'IM-IM': ((true_values >= insulin_sensitive_threshold) &
           (true_values <= insulin_resistance_threshold) &
           (predicted_values <= insulin_resistance_threshold) &
           (predicted_values >= insulin_sensitive_threshold)).sum(),
    'IS-IS': ((true_values <= insulin_sensitive_threshold) &
           (predicted_values <= insulin_sensitive_threshold)).sum(),
    'IR-IS': ((true_values >= insulin_resistance_threshold) &
           (predicted_values <= insulin_sensitive_threshold)).sum(),
  }

  # --- Add counts to the plot ---
  for i, (quadrant, count) in enumerate(quadrant_counts.items()):
      x_pos, y_pos = quadrant_coordinates[quadrant]
      plt.text(x_pos, y_pos, f'{quadrant}: {count}', fontsize=14,
              fontweight='bold', bbox=dict(facecolor='white', alpha=0.7))

  plt.legend(fontsize=10, loc="upper left")
  plt.tight_layout()
  plt.show()


def scater_plot_only(true_values, predicted_values):
  r2 = metrics.r2_score(true_values, predicted_values)
  insulin_sensitive_threshold = 1.5
  insulin_resistance_threshold = 2.9

  # Create the plot
  fig, ax = plt.subplots(figsize=(12, 10), dpi=720)

  # Scatter plot
  sns.scatterplot(x=true_values, y=predicted_values, s=70, alpha=0.4,
                  color='blue', edgecolor='black', linewidth=0.5)

  # Add horizontal and vertical lines (on top of the shaded regions)
  plt.axvline(x=insulin_sensitive_threshold, color='orange', linestyle=':',
              linewidth=5, label='Insulin Sensitive Threshold')
  plt.axvline(x=insulin_resistance_threshold, color='red',
              linestyle='--', linewidth=5, label='Insulin Resistance Threshold')
  plt.axhline(y=insulin_sensitive_threshold, color='orange', linestyle=':', linewidth=5)
  plt.axhline(y=insulin_resistance_threshold, color='red', linestyle='--', linewidth=5)
  plt.xlim(0, 7)
  plt.ylim(0, 7)
  # Enhance plot elements
  plt.xlabel("\nTrue HOMA-IR", fontsize=14, fontweight='bold')
  plt.ylabel("Predicted HOMA-IR", fontsize=14, fontweight='bold')
  plt.title(f"{exp_id_to_name_dict[experiment_name['name']]} \n "
            r"($R^2$ ="
            f"{r2:.2f})", fontsize=18, fontweight='bold')
  plt.xticks(fontsize=12)
  plt.yticks(fontsize=12)
  plt.grid(True, linestyle='--', alpha=0.5)
  # Define threshold values

  plt.legend(fontsize=10, loc="upper left")
  plt.tight_layout()
  plt.show()

In [ ]:
# @title Experiment Definitions

EXP_1 = {'name': "EXP_1",
'description': "Fitbit + Demographics",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std'],
'label': "homa_ir"}

EXP_2 = {'name': "EXP_2",
'description': "Fitbit subset + Demographics",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'STEPS_Daily_mean', 'SLEEP_Duration_mean'],
'label': "homa_ir"}

EXP_3 = {'name': "EXP_3",
'description': "Fitbit subset",
'features': ['participant_id',
'RHR_FreeLiving_mean', 'STEPS_Daily_mean', 'SLEEP_Duration_mean'],
'label': "homa_ir"}

EXP_4 = {'name': "EXP_4",
'description': "Demographics",
'features': ['participant_id', 'bmi','age'],
'label': "homa_ir"}

EXP_5 = {'name': "EXP_5",
'description': "Fitbit + Demographics + HbA1c",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'hba1c'],
'label': "homa_ir"}

EXP_6 = {'name': "EXP_6",
'description': "Demographics + HbA1c",
'features': ['participant_id', 'bmi','age',
'hba1c'],
'label': "homa_ir"}

EXP_7 = {'name': "EXP_7",
'description': "HbA1c",
'features': ['participant_id',
'hba1c'],
'label': "homa_ir"}

EXP_8 = {'name': "EXP_8",
'description': "Demographics + Fasting Glucose",
'features': ['participant_id', 'bmi','age',
'glucose'],
'label': "homa_ir"}

EXP_9 = {'name': "EXP_9",
'description': "Fasting Glucose",
'features': ['participant_id',
'glucose'],
'label': "homa_ir"}

EXP_10 = {'name': "EXP_10",
'description': "Fitbit + Demographics + Fasting Glucose",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'glucose'],
'label': "homa_ir"}

EXP_11 = {'name': "EXP_11",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_12 = {'name': "EXP_12",
'description': "Demographics + Lipid Panel + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_13 = {'name': "EXP_13",
'description': "Demographics + Hypertension",
'features': ['participant_id', 'bmi','age',
'hypertension_binary'],
'label': "homa_ir"}

EXP_14 = {'name': "EXP_14",
'description': "Fitbit + Demographics + Hypertension",
'features': ['participant_id', 'bmi','age',
'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std'],
'label': "homa_ir"}

EXP_15 = {'name': "EXP_15",
'description': "Fitbit + Demographics + Fasting Glucose + Hypertension",
'features': ['participant_id', 'bmi','age', 'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'glucose'],
'label': "homa_ir"}


EXP_16 = {'name': "EXP_16",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel + Hypertension",
'features': ['participant_id', 'bmi','age', 'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_17 = {'name': "EXP_17",
'description': "Lipid Panel + Metabolic Panel",
'features': ['participant_id',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_18 = {'name': "EXP_18",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel + HbA1c",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt',
'hba1c'],
'label': "homa_ir"}


EXP_19 = {'name': "EXP_19",
'description': "Lipid Panel",
'features': ['participant_id',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_20 = {'name': "EXP_20",
'description': "Metabolic Panel",
'features': ['participant_id', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_21 = {'name': "EXP_21",
'description': "Fitbit + Demographics + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_22 = {'name': "EXP_22",
'description': "Fitbit + Demographics + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}



EXP_23 = {'name': "EXP_23",
'description': "Demographics + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}

EXP_24 = {'name': "EXP_24",
'description': "Demographics + Metabolic Panel",
'features': ['participant_id', 'bmi','age', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_25 = {'name': "EXP_25",
'description': "Fasting Insulin",
'features': ['participant_id', 'insulin'],
'label': "homa_ir"}

EXP_26 = {'name': "EXP_26",
'description': "Fasting Glucose + Fasting Insulin",
'features': ['participant_id', 'glucose', 'insulin'],
'label': "homa_ir"}


EXP_27 = {'name': "EXP_27",
'description': "Fitbit + Demographics + Fasting Glucose + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std', 'glucose',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_28 = {'name': "EXP_28",
'description': "Fitbit",
'features': ['participant_id',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',],
'label': "homa_ir"}

In [ ]:
exp_id_to_name_dict = {"EXP_26": "Fasting Glucose + Fasting Insulin (Trivial)",
                       "EXP_25": "Fasting Insulin (Control)",
                       "EXP_9": "Fasting Glucose",
                       "EXP_7": "HbA1c",
                       "EXP_19": "Lipid Panel",
                       "EXP_20": "Metabolic Panel",
                       "EXP_17": "Lipid Panel + Metabolic Panel",
                       "EXP_4": "Demo.",
                       "EXP_6": "Demo.+ HbA1c",
                       "EXP_8": "Demo. + Glucose",
                       "EXP_12": "Demo. + Lipid Panel + Metabolic Panel",
                       "EXP_13": "Demo. + Hypertension",
                       "EXP_23": "Demo. + Lipid Panel",
                       "EXP_24": "Demo. + Metabolic Panel",
                       "EXP_3": "Wearables Subset",
                       "EXP_1": "Wearables + Demo.",
                       "EXP_2": "Fitbit Subset + Demo.",
                       "EXP_5": "Wearables + Demo. + HbA1c",
                       "EXP_10": "Wearables + Demo. + Glucose",
                       "EXP_11": "Wearables + Demo. + Lipid Panel + Metabolic Panel",
                       "EXP_14": "Wearables + Demo. + Hypertension",
                       "EXP_15": "Wearables + Demo. + Glucose + Hypertension",
                       "EXP_16": "Wearables + Demo. + Lipid Panel + Metabolic Panel + Hypertension",
                       "EXP_18": "Wearables + Demo. + Lipid Panel + Metabolic Panel + HbA1c",
                       "EXP_21": "Wearables + Demo. +Lipid Panel",
                       "EXP_22": "Wearables + Demo. + Metabolic Panel",
                       "EXP_27": "Wearables + Demo. + Glucose + Lipid Panel",
                       "EXP_28": "Wearables",
                       }

In [ ]:
experiment_to_latent_space = {}
experiment_to_prediction_df = {}

for experiment_name in [EXP_1, EXP_21, EXP_10, EXP_11]:

  learner = "l1-l2"
  n_days = 120
  prediction_df = pd.DataFrame()
  random_seeds=[0, 92, 1, 2024, 12121]
  use_mae = True
  ae_type = "masked_autoencoder"
  latent_space = 64
  device = "cuda"
  feature_cols = experiment_name["features"]
  target_col = experiment_name["label"]

  name = experiment_name['name']
  path_to_load = os.path.join(AUTOENCODER_STUDY_RESULTS_FOLDER, f"ablation_models/masked_autoencoder_{name}_kfold_n_days_120/")


  if learner == "l1-l2":
        kfold_found_params = {'booster': 'gblinear',
                              'learning_rate': 0.19,
                              'n_estimators': 50,
                              'eta':0.1,
                              'reg_alpha': 0,
                              'reg_lambda': 0}

  elif learner == "tree":
        kfold_found_params = {
            "booster": "gbtree",
            "learning_rate": 0.1,
            "max_depth": 1,
            "n_estimators": 85,
            "reg_alpha": 2,
            "reg_lambda": 2,
        }
  else:
    raise ValueError(
        f"Unknown learner: {learner}. Learners must be 'l1-l2' or 'tree'."
    )

  # Get the training data
  training_df = training_data_aggregated_all_days.copy()
  training_df = training_data_aggregated_all_days.loc[:, feature_cols]
  training_df[target_col] =  training_data_aggregated_all_days[target_col]
  participant_ids = training_df['participant_id'].values
  accuracy_dfs = []
  ordered_participant_ids = []
  all_predictions = []
  all_true_labels = []
  clf_labels_preds = []
  clf_labels_true_labels = []
  cv = model_selection.KFold(n_splits=5)

  cv_iterator = tqdm(
      cv.split(training_df),
      total=5,
      desc=f"  {experiment_name['name']}",
      leave=True,
  )

  # Perform cross-validation
  all_x_test_encoded = []
  for i, (train_index, test_index) in enumerate(cv_iterator):

    np.random.seed(random_seeds[i])
    torch.manual_seed(random_seeds[i])

    # save participant IDs
    participant_ids_train = participant_ids[train_index]
    participant_ids_test = participant_ids[test_index]
    X_train = training_df.iloc[train_index].drop(columns=["participant_id",
                                                          target_col], axis=1)
    y_train = training_df.iloc[train_index][target_col]
    X_test = training_df.iloc[test_index].drop(columns=["participant_id",
                                                        target_col], axis=1)
    y_test = training_df.iloc[test_index][target_col]
    X_train, X_test = preprocess_data(X_train, X_test.values)
    dl_model = MaskedAutoencoder(input_dim=X_train.shape[1],
                                        latent_space=latent_space).to(device)

    name = experiment_name['name']
    with open(
        os.path.join(path_to_load, f"kfold_iteration_{i}_masked_autoencoder_{name}_n_days_120.pth"),
        "rb+") as fd:

      dl_model.load_state_dict(torch.load(fd))
    # Get encoded representations
    dl_model.eval()
    with torch.no_grad():
      X_train_encoded, _ = dl_model(torch.Tensor(X_train).to(device),
                                      return_encoded = True)
      X_test_encoded, _ = dl_model(torch.Tensor(X_test).to(device),
                                    return_encoded = True)

    # Convert to numpy for XGBoost
    X_train = X_train_encoded.detach().cpu().numpy()
    X_test = X_test_encoded.detach().cpu().numpy()

    # Initialize and train the XGBoost model
    xgb_regressor = xgboost.XGBRegressor(
        **kfold_found_params, random_state=random_seeds[i]
    )
    xgb_regressor.fit(X_train, y_train.values)
    predicted_labels = xgb_regressor.predict(X_test)

    # Store predictions and true labels for later evaluation
    all_true_labels.append(y_test.values)
    all_predictions.append(predicted_labels)
    ordered_participant_ids.append(participant_ids_test)

  experiment_to_latent_space[experiment_name['name']] = all_x_test_encoded
  true_values = np.concatenate(all_true_labels)
  predicted_values = np.concatenate(all_predictions)
  participant_ids_predicted = np.concatenate(ordered_participant_ids)

  prediction_df['participant_id'] = participant_ids_predicted
  prediction_df['predicted_values'] = predicted_values
  prediction_df['true_values'] = true_values
  prediction_df['Experiment_ID'] = experiment_name['name']

  experiment_to_prediction_df[experiment_name['name']] = prediction_df
  scater_plot_only(true_values, predicted_values)
  # To make things easier and consistent across the plots, we use the values of
  # computed R^2 (as provided in another notebook) to add to the plotting title
  match experiment_name['name']:
    case 'EXP_1':
      current_r2 = 0.2189654744
    case 'EXP_10':
      current_r2 = 0.4356375829
    case 'EXP_21':
      current_r2 = 0.3133970078
    case 'EXP_11':
      current_r2 = 0.4953900592
  plot_with_counts_on_quadrants(true_values, predicted_values,
                                current_r2_score = current_r2)


Output hidden; open in https://colab.research.google.com to view.

## Getting and Storing Participant ID, Predictions, True Values and Experiment IDs as Dataframes

In [ ]:
experiment_to_latent_space = {}
experiment_to_prediction_df = {}

for experiment_name in [EXP_1, EXP_2, EXP_3, EXP_4, EXP_5, EXP_6, EXP_7, EXP_8,
                        EXP_9, EXP_10, EXP_11, EXP_12, EXP_13, EXP_14, EXP_15,
                        EXP_16, EXP_17, EXP_18, EXP_19, EXP_20, EXP_21, EXP_22,
                        EXP_23, EXP_24, EXP_25, EXP_26, EXP_27, EXP_28,]:

  learner = "l1-l2"
  n_days = 120
  prediction_df = pd.DataFrame()
  random_seeds=[0, 92, 1, 2024, 12121]
  use_mae = True
  ae_type = "masked_autoencoder"
  latent_space = 64
  device = "cuda"
  feature_cols = experiment_name["features"]
  target_col = experiment_name["label"]

  path_to_load = os.path.join(AUTOENCODER_STUDY_RESULTS_FOLDER,
                  f"ablation_models/masked_autoencoder_{experiment_name['name']}_kfold_n_days_120/")


  if learner == "l1-l2":
        kfold_found_params = {'booster': 'gblinear',
                              'learning_rate': 0.19,
                              'n_estimators': 50,
                              'eta':0.1,
                              'reg_alpha': 0,
                              'reg_lambda': 0}

  elif learner == "tree":
    kfold_found_params = {
        "booster": "gbtree",
        "learning_rate": 0.1,
        "max_depth": 1,
        "n_estimators": 85,
        "reg_alpha": 2,
        "reg_lambda": 2,
    }
  else:
    raise ValueError(
        f"Unknown learner: {learner}. Learners must be 'l1-l2' or 'tree'."
    )

  # Get the training data
  training_df = training_data_aggregated_all_days.copy()
  training_df = training_data_aggregated_all_days.loc[:, feature_cols]
  training_df[target_col] =  training_data_aggregated_all_days[target_col]
  participant_ids = training_df['participant_id'].values
  accuracy_dfs = []
  ordered_participant_ids = []
  all_predictions = []
  all_true_labels = []
  clf_labels_preds = []
  clf_labels_true_labels = []
  all_fold_ids = []
  cv = model_selection.KFold(n_splits=5)

  cv_iterator = tqdm(
      cv.split(training_df),
      total=5,
      desc=f"  {experiment_name['name']}",
      leave=True,
  )

  # Perform cross-validation
  all_x_test_encoded = []
  for i, (train_index, test_index) in enumerate(cv_iterator):

    np.random.seed(random_seeds[i])
    torch.manual_seed(random_seeds[i])

    # save participant IDs
    participant_ids_train = participant_ids[train_index]
    participant_ids_test = participant_ids[test_index]
    X_train = training_df.iloc[train_index].drop(columns=["participant_id",
                                                          target_col], axis=1)
    y_train = training_df.iloc[train_index][target_col]
    X_test = training_df.iloc[test_index].drop(columns=["participant_id",
                                                        target_col], axis=1)
    y_test = training_df.iloc[test_index][target_col]
    X_train, X_test = preprocess_data(X_train, X_test.values)
    dl_model = MaskedAutoencoder(input_dim=X_train.shape[1],
                                        latent_space=latent_space).to(device)
    with open(
        os.path.join(path_to_load,
        f"kfold_iteration_{i}_masked_autoencoder_{experiment_name['name']}_n_days_120.pth")
        , "rb+") as fd:

      dl_model.load_state_dict(torch.load(fd))
    # Get encoded representations
    dl_model.eval()
    with torch.no_grad():
      X_train_encoded, _ = dl_model(torch.Tensor(X_train).to(device),
                                      return_encoded = True)
      X_test_encoded, _ = dl_model(torch.Tensor(X_test).to(device),
                                    return_encoded = True)

    # Convert to numpy for XGBoost
    X_train = X_train_encoded.detach().cpu().numpy()
    X_test = X_test_encoded.detach().cpu().numpy()

    # Initialize and train the XGBoost model
    xgb_regressor = xgboost.XGBRegressor(
        **kfold_found_params, random_state=random_seeds[i]
    )
    xgb_regressor.fit(X_train, y_train.values)
    predicted_labels = xgb_regressor.predict(X_test)

    # Store predictions and true labels for later evaluation
    all_true_labels.append(y_test.values)
    all_predictions.append(predicted_labels)
    fold_id = [f"Fold_{i}"] * len(y_test)
    all_fold_ids.append(fold_id)
    ordered_participant_ids.append(participant_ids_test)

  experiment_to_latent_space[experiment_name['name']] = all_x_test_encoded
  true_values = np.concatenate(all_true_labels)
  predicted_values = np.concatenate(all_predictions)
  participant_ids_predicted = np.concatenate(ordered_participant_ids)

  prediction_df['participant_id'] = participant_ids_predicted
  prediction_df['predicted_values'] = predicted_values
  prediction_df['true_values'] = true_values
  prediction_df['Experiment_ID'] = experiment_name['name']
  prediction_df['fold_id'] = np.concatenate(all_fold_ids).astype(str)

  experiment_to_prediction_df[experiment_name['name']] = prediction_df

  EXP_28: 100%|██████████| 5/5 [00:00<00:00, 18.74it/s]


In [ ]:
def mcnemar_test(model1_preds, model2_preds, true_labels):
  """
  Perform McNemar's test to compare the performance of two models.

  Args:
    model1_preds (array-like): Predicted binary labels from the first model.
    model2_preds (array-like): Predicted binary labels from the second model.
    true_labels (array-like): True binary labels.

  Returns:
    float: p-value from McNemar's test.
  """
  # Create a contingency table
  contingency_table = np.zeros((2, 2))
  for i in range(len(true_labels)):
    if model1_preds[i] == true_labels[i] and model2_preds[i] == true_labels[i]:
      contingency_table[0, 0] += 1
    elif model1_preds[i] != true_labels[i] and model2_preds[i] == true_labels[i]:
      contingency_table[0, 1] += 1
    elif model1_preds[i] == true_labels[i] and model2_preds[i] != true_labels[i]:
      contingency_table[1, 0] += 1
    else:
      contingency_table[1, 1] += 1

  # Perform McNemar's test
  result = mcnemar(contingency_table, exact=False)

  return result.pvalue

n_models = len(experiment_to_prediction_df)
p_values = np.zeros((n_models, n_models))

for i in range(n_models):
  model1_name = list(experiment_to_prediction_df.keys())[i]
  for j in range(i + 1, n_models):
    model2_name = list(experiment_to_prediction_df.keys())[j]

    model1_preds = (experiment_to_prediction_df[model1_name]['predicted_values'].values >= 2.9).astype(int)
    model2_preds = (experiment_to_prediction_df[model2_name]['predicted_values'].values >= 2.9).astype(int)
    # True values are the same across all experiments
    true_labels = (experiment_to_prediction_df[model1_name]['true_values'].values >= 2.9).astype(int)
    p_value = mcnemar_test(model1_preds, model2_preds, true_labels)

    p_values[i, j] = p_value


# Flatten the upper triangle of the p-values matrix
p_values_flat = p_values[np.triu_indices_from(p_values, k=1)]

# Adjust p-values for multiple comparisons using Benjamini-Hochberg correction
reject, p_values_adj, _, _ = multipletests(p_values_flat, method='fdr_bh')

# Reshape adjusted p-values back to the original matrix shape
p_values_adj_matrix = np.zeros((n_models, n_models))
p_values_adj_matrix[np.triu_indices_from(p_values_adj_matrix, k=1)] = p_values_adj


In [ ]:
adjusted_p_values_df = pd.DataFrame(p_values_adj_matrix,
                                    index=list(experiment_to_prediction_df.keys()),
                                    columns=list(experiment_to_prediction_df.keys()))

adjusted_p_values_df

,EXP_1,EXP_2,EXP_3,EXP_4,EXP_5,EXP_6,EXP_7,EXP_8,EXP_9,EXP_10,...,EXP_19,EXP_20,EXP_21,EXP_22,EXP_23,EXP_24,EXP_25,EXP_26,EXP_27,EXP_28
EXP_1,0.0,0.017186,0.000136,0.000564,1.084274e-01,0.154238,1.000000,0.822138,0.775647,7.380387e-03,...,5.293437e-02,0.225239,0.108419,5.090747e-03,0.647595,9.159262e-03,3.126853e-21,9.206297e-19,7.139784e-05,1.093599e-05
EXP_2,0.0,0.000000,0.021267,0.157285,5.091050e-04,0.958344,0.199616,0.335161,0.129801,1.850563e-05,...,4.983243e-01,0.012285,0.000749,1.599006e-05,0.414618,6.482672e-05,2.715163e-25,4.910454e-23,7.157629e-08,6.602221e-03
EXP_3,0.0,0.000000,0.000000,0.775647,9.801077e-07,0.113997,0.003898,0.019777,0.002396,1.590761e-08,...,5.330866e-01,0.000041,0.000001,1.446853e-08,0.024348,1.795164e-07,1.176214e-30,7.669489e-28,1.770758e-11,5.288634e-01
EXP_4,0.0,0.000000,0.000000,0.000000,3.441377e-06,0.073503,0.015445,0.011845,0.006995,3.569696e-08,...,7.797005e-01,0.000277,0.000005,3.360928e-08,0.009804,8.060760e-09,5.716310e-32,1.151190e-28,9.151257e-11,5.330866e-01
EXP_5,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000802,0.246295,0.154048,0.540378,2.055305e-01,...,2.250127e-03,0.865176,0.951933,1.490212e-01,0.091897,1.757221e-01,1.720512e-16,5.795699e-15,7.248311e-03,7.427502e-08
EXP_6,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.209931,0.318061,0.133848,7.737494e-05,...,4.370430e-01,0.011916,0.006258,3.755079e-05,0.451956,3.790932e-05,4.092272e-25,1.720846e-23,4.388926e-07,5.779379e-02
EXP_7,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.809632,0.761016,3.216995e-02,...,4.430265e-02,0.179808,0.292993,1.640810e-02,0.673578,2.006012e-02,2.443283e-17,3.907250e-16,5.347840e-04,9.140502e-04
EXP_8,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.479996,5.091050e-04,...,1.007280e-01,0.073503,0.141731,6.658514e-04,0.897725,5.298342e-04,1.014054e-21,7.135957e-23,6.549577e-06,7.983894e-03
EXP_9,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,3.344659e-02,...,2.075970e-02,0.227198,0.502642,2.120145e-02,0.470261,2.132952e-02,1.789006e-16,2.427389e-16,6.860197e-04,6.658514e-04
EXP_10,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,5.937291e-05,0.388463,0.310025,8.407087e-01,0.006538,8.977252e-01,2.696629e-12,3.766873e-12,1.054288e-01,9.505791e-10
